In [47]:
import plotly
plotly.tools.set_credentials_file(username='anakai3@berkeley.edu', api_key='OOCaTL37ib8u0gwsCQOl')
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
from ipywidgets import widgets 
from IPython.display import display
from plotly.graph_objs import *
from plotly.widgets import GraphWidget
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
mapbox_access_token ='pk.eyJ1IjoiYW5ha2FpMyIsImEiOiJjamZsbm53dTUwanljMndzMnR0ZXZiOWUzIn0.tJZ5UpqBKnHl-TbScs7c5A'

#Air pollution Data
#Adding Year Column
df2=pd.read_csv('pollutiontest.csv')
df2['year'] = pd.DatetimeIndex(df2['Date_Local']).year
df_airpol_agg=df2.groupby(['City'],as_index=False)['co_aqi_level','so2_aqi_level','no2_aqi_level','o3_aqi_level'].mean()
df_airpol_agg=df_airpol_agg.drop([21])
df_airpol_agg=df_airpol_agg.round(2)
df_airpol_agg=df_airpol_agg.reset_index(drop=True)


#Using GeoPy API to get Lat Longs of the Cities 

# city_list=list(df_airpol_agg.City.unique())
# from geopy.geocoders import Nominatim
# geolocator = Nominatim()
# city_dict={}
# for c in city_list:
#     try:
#         location = geolocator.geocode(str(c)+',CA')
#         city_dict[c]=(location.latitude,location.longitude)
#     except:
#         city_dict[c]=('NA','NA')
# city_dict['San Pablo']=(37.9621, -122.3455)

city_dict={'Arden-Arcade': (38.6039613, -121.383003529994),
 'Bakersfield': (35.3738712, -119.0194639),
 'Benicia': (38.049365, -122.1585777),
 'Berkeley': (44.3669375, -80.7272504),
 'Bethel Island': (38.0149216, -121.6405085),
 'Burbank': (34.1816482, -118.3258554),
 'Calexico': (32.6790572, -115.498703),
 'Capitan': (32.9164928, -116.8296446),
 'Chula Vista': (32.6400541, -117.0841955),
 'Concord': (45.452798, -62.7905136),
 'Costa Mesa': (33.6633386, -117.903317),
 'Crockett': (38.0525612, -122.2202603),
 'Davenport': (51.04172695, -114.085008927677),
 'Eureka': (79.98855485, -85.9373384357636),
 'Fontana': (34.0922335, -117.435048),
 'Fresno': (36.7378, -119.7871),
 'Goleta': (34.4358295, -119.8276389),
 'Hawthorne': (45.9647141, -61.4706939),
 'Lompoc': (34.6391501, -120.4579409),
 'Long Beach': (44.4567705, -78.7359702),
 'Los Angeles': (34.054935, -118.244476),
 'Oakland': (43.0317491, -80.3310713),
 'Pittsburg': (38.0181745, -121.8901232),
 'Rubidoux': (33.9964933, -117.4079318),
 'San Diego': (32.7174209, -117.1627714),
 'San Francisco': (46.8443248, -71.2743272),
 'San Pablo': (37.9621, -122.3455),
 'Vallejo': (38.1040864, -122.2566367),
 'Vandenberg Air Force Base': (34.7088437, -120.544742203778),
 'Victorville': (34.5361067, -117.2911565),
 'West Los Angeles': (34.0463986, -118.448135)}

#Adding Lat Longs to Data Frame
df_airpol_agg['Latitude']=100
df_airpol_agg['Longitude']=100
cnt=0
for i in city_dict:
    df_airpol_agg.loc[cnt, 'Latitude'] = city_dict[df_airpol_agg.City[cnt]][0]
    df_airpol_agg.loc[cnt, 'Longitude'] = city_dict[df_airpol_agg.City[cnt]][1]
    cnt+=1
    

#Making Plotly ScatterMapBox For Air Pollution Data
data=[]
metric_list=['so2_aqi_level','no2_aqi_level','o3_aqi_level']

for i in metric_list:
    data.append(
    Scattermapbox(
        lat=df_airpol_agg['Latitude'],
        lon=df_airpol_agg['Longitude'],
        mode='markers',
        marker=Marker(
            size=8,
            color= df_airpol_agg[i],
            colorscale = 'Reds',
            opacity=1.0
        ),
        text=df_airpol_agg[i],
        hoverinfo='text'
    ))
nested_vis_list=[]
for i in range(0,3):
    nested_vis_list.append([0]*3)
    nested_vis_list[i][i]=1
for i in range(0,3):
    for j in range(0,3):
        if nested_vis_list[i][j]==1:
            nested_vis_list[i][j]=True
        else:
            nested_vis_list[i][j]=False          
update_list=[]
cnt=0
for i in metric_list:
    update_list.append(dict(label = i,method = 'update',
                            args = [{'visible': nested_vis_list[cnt]},{'title': i,'annotations': []}]))
    cnt+=1

updatemenus = list([
    dict(active=-1,
         buttons=update_list,
    )
])
        
layout = Layout(
    title='Air Pollution API Levels in California',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    updatemenus=updatemenus,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=37,
            lon=-118
        ),
        pitch=0,
        zoom=4,
        style='light'
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig)